In [9]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3:70b", base_url = 'http://172.28.105.30/backend', format='json')

In [6]:
from pydantic import BaseModel, Field
from typing import List

class Solution(BaseModel):
    temperature: float = Field(..., description="The temperature of the solution creation")
    atmosphere: str = Field(..., description="The atmosphere of the solution creation")
    method: str = Field(..., description="The method of the solution creation")
    time: float = Field(..., description="The time needed for the solution creation")
    solutes: List[str] = Field(..., description="The solutes used in the solution")
    solute_masses: List[float] = Field(..., description="The masses in miligramm of the solutes used in the solution")
    solvents: List[str] = Field(..., description="The solvents used in the solution")
    solvent_volumes: List[float] = Field(...,
                                         description="The volumes in mililiter of the solvents used in the solution")

schema_solution = Solution.schema()

print(schema_solution)


{'properties': {'temperature': {'description': 'The temperature of the solution creation', 'title': 'Temperature', 'type': 'number'}, 'atmosphere': {'description': 'The atmosphere of the solution creation', 'title': 'Atmosphere', 'type': 'string'}, 'method': {'description': 'The method of the solution creation', 'title': 'Method', 'type': 'string'}, 'time': {'description': 'The time needed for the solution creation', 'title': 'Time', 'type': 'number'}, 'solutes': {'description': 'The solutes used in the solution', 'items': {'type': 'string'}, 'title': 'Solutes', 'type': 'array'}, 'solute_masses': {'description': 'The masses in miligramm of the solutes used in the solution', 'items': {'type': 'number'}, 'title': 'Solute Masses', 'type': 'array'}, 'solvents': {'description': 'The solvents used in the solution', 'items': {'type': 'string'}, 'title': 'Solvents', 'type': 'array'}, 'solvent_volumes': {'description': 'The volumes in mililiter of the solvents used in the solution', 'items': {'

In [7]:
text = "Solution Preparation: In this study, we prepared a solution for catalyst synthesis. First, the solutes copper nitrate and zinc chloride were accurately weighed (500.0 mg and 200.0 mg, respectively) using a high-precision balance. The solvents Water and Acetone were then added to the solutes in specific volumes (50.0 mL and 20.0 mL, respectively) in a glass vial. The mixture was stirred for 30 minutes at room temperature (25.0°C) under an inert gas atmosphere to ensure complete dissolution of the solutes. Finally, the solvent was evaporated using a rotary evaporator at reduced pressure, resulting in a solution with the desired composition. This process took approximately 2.5 hours to complete. The prepared solution was then used for further experimentation and analysis."



In [12]:
message = "Extract the information of the following text "+":\n"+text+"\n\n using the following schema:\n"+str(schema_solution)+"\n Return a JSON object with the extracted information." 
response=llm.invoke(message)

print(response)

{
    "title": "Solution",
    "type": "object",
    "properties": {
        "temperature": {"description": "The temperature of the solution creation", "title": "Temperature", "type": "number", "value": 25.0},
        "atmosphere": {"description": "The atmosphere of the solution creation", "title": "Atmosphere", "type": "string", "value": "inert gas"},
        "method": {"description": "The method of the solution creation", "title": "Method", "type": "string", "value": "stirring and evaporation"},
        "time": {"description": "The time needed for the solution creation", "title": "Time", "type": "number", "value": 2.5},
        "solutes": {"description": "The solutes used in the solution", "title": "Solutes", "type": "array", "value": ["copper nitrate", "zinc chloride"]},
        "solute_masses": {"description": "The masses in miligramm of the solutes used in the solution", "title": "Solute Masses", "type": "array", "value": [500.0, 200.0]},
        "solvents": {"description": "The s

In [15]:
###another example with a different result, using chaining according to example from https://python.langchain.com/docs/integrations/chat/ollama/

import json

from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(model="llama3:70b", format="json", temperature=0)
llm.base_url = 'http://172.28.105.30/backend'

messages = [
    HumanMessage(
        content="Please tell me about a synthesis step using the following JSON schema:"
    ),
    HumanMessage(content="{dumps}"),
    HumanMessage(
        content="Now, considering the schema, extract the information from the following text: "
    ),
    HumanMessage(
        content=text),
]

prompt = ChatPromptTemplate.from_messages(messages)
dumps = json.dumps(schema_solution, indent=2)

chain = prompt | llm | StrOutputParser()

print(chain.invoke({"text": text, "dumps": dumps}))

{ 
    "Synthesis Step": {
        "Step Name": "Solution Preparation",
        "Solutes": [
            {"Name": "Copper Nitrate", "Weight (mg)": 500.0},
            {"Name": "Zinc Chloride", "Weight (mg)": 200.0}
        ],
        "Solvents": [
            {"Name": "Water", "Volume (mL)": 50.0},
            {"Name": "Acetone", "Volume (mL)": 20.0}
        ],
        "Conditions": {
            "Temperature (°C)": 25.0,
            "Atmosphere": "Inert Gas"
        },
        "Procedure": [
            {"Action": "Stirring", "Duration (minutes)": 30},
            {"Action": "Evaporation", "Method": "Rotary Evaporator", "Pressure": "Reduced"}
        ],
        "Time Required (hours)": 2.5
    }
} 
   


   


   


   


   


   


   


   


   


   


   


   


   


   


   



